In [2]:
import pandas as pd
import numpy as np
import os, sys
import math
import copy
from copy import deepcopy

## PROCESS FIRSt SAMPLE

In [2]:
input = '''broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a'''

input=input.split('\n')

In [107]:
input = '''broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output'''

input=input.split('\n')

In [5]:
def build_module_dict(input):

    module_lib = {}

    for line in input:
        module, destination = line.split(' -> ')
        dest_list = destination.split(', ')
        if module == 'broadcaster':
            module_name = module
            module_type = module
        else:
            module_type, module_name = module[0], module[1:]

        for dest_module in dest_list:
            try:
                module_lib[dest_module]['sources'].append(module_name)
            except KeyError:
                try:
                    module_lib[dest_module]['sources']=[module_name]
                except KeyError:
                    module_lib[dest_module] = {'sources': [module_name]}

        try:
            module_lib[module_name]['type'] = module_type
            module_lib[module_name]['destinations'] = dest_list
        except KeyError:
            module_lib[module_name] = {'type':module_type, 'destinations':dest_list}

        if module_type == '%':
            module_lib[module_name]['data'] = False
        
    for module_name, module in module_lib.items():
        try:
            module_type = module['type']
        except KeyError:
            module_lib[module_name]['type'] = 'output'
            continue

        if module_type == '&':
            try:
                source_list = module['sources']
            except KeyError:
                source_list = []

            module_lib[module_name]['data'] = {orig_module: False for orig_module in source_list}
    
    return module_lib

In [109]:
module_lib_backup = build_module_dict(input)

In [6]:
def process(pulse_task):
    pulse, module_name, originator = pulse_task
    
    try:
        module = module_lib[module_name]
    except KeyError:
        return None
    
    if module['type'] == 'broadcaster':
        return (pulse, module['destinations'], module_name)
    
    if module['type'] == '%':
        if not pulse:
            module['data'] = not(module['data'])
            module_lib[module_name] = module
            return (module['data'], module['destinations'], module_name)
        
    if module['type'] == '&':
        module['data'][originator] = pulse
        module_lib[module_name] = module
        return (not(all(signal for signal in module['data'].values())), module['destinations'], module_name)
    
    return None

In [111]:
from collections import deque

module_lib = deepcopy(module_lib_backup)

num_lows = 0
num_highs = 0
for i in range(1000):
    #print('-----------------------------------------------------')
    pulse_list = deque([(False, 'broadcaster', 'button')])

    while pulse_list:
        pulse_task = pulse_list.popleft()
        pulse, dest, orig = pulse_task
        #print(f'Sending pulse {pulse} from {orig} to {dest}')
        if pulse_task[0]:
            num_highs += 1
        else:
            num_lows += 1
        output = process(pulse_task)
        if output:
            pulse, destination_list, originator = output
            for destination in destination_list:
                pulse_list.append((pulse, destination, originator))

num_lows*num_highs, num_lows, num_highs

(11687500, 4250, 2750)

## PROCESS PART 1

In [7]:
# READ FILE
inputfilepath = './input.txt'

input = []
with open(inputfilepath,'r') as f:
    for line in f:
        input.append(line.rstrip())

In [8]:
module_lib_backup = build_module_dict(input)

In [116]:
module_lib_backup

{'hq': {'sources': ['nr'],
  'type': '%',
  'destinations': ['ft', 'lh'],
  'data': False},
 'nr': {'type': '%',
  'destinations': ['hq'],
  'data': False,
  'sources': ['sz', 'ft']},
 'sn': {'sources': ['xk', 'lk'],
  'type': '%',
  'destinations': ['lp'],
  'data': False},
 'xk': {'type': '%',
  'destinations': ['sn'],
  'data': False,
  'sources': ['cl', 'lk']},
 'cl': {'type': '%',
  'destinations': ['xk'],
  'data': False,
  'sources': ['gm', 'lk']},
 'dq': {'sources': ['mj'],
  'type': '%',
  'destinations': ['qr', 'rv'],
  'data': False},
 'qr': {'sources': ['mj', 'rv', 'vl', 'jj', 'cz', 'dq', 'bz', 'mb', 'zj'],
  'type': '&',
  'destinations': ['cz', 'sp', 'lb', 'xt', 'fx'],
  'data': {'mj': False,
   'rv': False,
   'vl': False,
   'jj': False,
   'cz': False,
   'dq': False,
   'bz': False,
   'mb': False,
   'zj': False}},
 'mj': {'type': '%',
  'destinations': ['dq', 'qr'],
  'data': False,
  'sources': ['jj']},
 'lk': {'sources': ['gm', 'qm', 'vg', 'db', 'mh', 'sb', 'ks'],

In [117]:
from collections import deque

module_lib = deepcopy(module_lib_backup)

num_lows = 0
num_highs = 0
for i in range(1000):
    pulse_list = deque([(False, 'broadcaster', 'button')])

    while pulse_list:
        pulse_task = pulse_list.popleft()
        pulse, dest, orig = pulse_task
        #print(f'Sending pulse {pulse} from {orig} to {dest}')
        if pulse_task[0]:
            num_highs += 1
        else:
            num_lows += 1
        output = process(pulse_task)
        if output:
            pulse, destination_list, originator = output
            for destination in destination_list:
                pulse_list.append((pulse, destination, originator))

num_lows*num_highs

873301506

## PROCESS PART 2

A visual plot shows the graph is formed of 4 blocks consisting of 12 chained inversors '%' connected to a central accumulator '&' each.
These 4 blocks originate from the broadcaster, and deliver the accumulator output to a 4-input accumulator output system that will get to the output.

The connections are IN or OUT for every inversor except the first which is bidirectional. Chained inversors behave like a binary count (2 cycles in n-1 increase the n, 2 cycles in n will increase n+1).
The accumulator acts like a reset: whenever every IN inversor has a 1, it will add 1 to every OUT inversor, thus resetting the whole binary count to 0.

The final output will be obtained when all 4 blocks have an output of 1, which happens at their reset cycles.

So we need to find the reset cycles for each block, multiply all the different numbers found, and that will be the result.


In [36]:
for name, module in module_lib_backup.items():
    try:
        if (module['type']) == '&':
            print(name)
            print(module)
    except KeyError:
        pass

qr
{'sources': ['mj', 'rv', 'vl', 'jj', 'cz', 'dq', 'bz', 'mb', 'zj'], 'type': '&', 'destinations': ['cz', 'sp', 'lb', 'xt', 'fx'], 'data': {'mj': False, 'rv': False, 'vl': False, 'jj': False, 'cz': False, 'dq': False, 'bz': False, 'mb': False, 'zj': False}}
lk
{'sources': ['gm', 'qm', 'vg', 'db', 'mh', 'sb', 'ks'], 'type': '&', 'destinations': ['sn', 'cc', 'xk', 'rn', 'gm', 'cl', 'lp'], 'data': {'gm': False, 'qm': False, 'vg': False, 'db': False, 'mh': False, 'sb': False, 'ks': False}}
lz
{'sources': ['mv', 'ts', 'xc', 'nk', 'vb', 'hd', 'xl', 'dv'], 'type': '&', 'destinations': ['gx', 'xn', 'jq', 'fb', 'ts', 'rr'], 'data': {'mv': False, 'ts': False, 'xc': False, 'nk': False, 'vb': False, 'hd': False, 'xl': False, 'dv': False}}
sp
{'sources': ['qr'], 'type': '&', 'destinations': ['dd'], 'data': {'qr': False}}
dd
{'type': '&', 'destinations': ['rx'], 'sources': ['nx', 'sp', 'cc', 'jq'], 'data': {'nx': False, 'sp': False, 'cc': False, 'jq': False}}
ft
{'sources': ['jn', 'hq', 'sz', 'fr',

In [109]:
chains = ['101101011111', '100001011111', '110100001111', '11100010111']
chain_nums = []
total_val = 1


def binary_to_int(binary_string):
    cumulator = 0
    bitpower = 1
    for bit in binary_string:
        cumulator += bitpower*int(bit)
        bitpower *= 2
    return cumulator

In [107]:
def get_accumulator(thread):
    module = module_lib_backup[thread]
    if (module['type'] == '&'):
        return thread
    destinations = module['destinations']
    for destination in destinations:
        accumulator = get_accumulator(destination)
        if accumulator:
            return accumulator
    return None

def get_thread_binary_representation(thread, accumulator):
    binary_string = []
    module = module_lib_backup[thread]
    if not(module['type'] == '%'):
        return binary_string
    
    destinations = module['destinations']
    if accumulator in destinations:
        binary_string.append('1')
    else:
        binary_string.append('0')
    
    for destination in destinations:
        binary_string += get_thread_binary_representation(destination, accumulator)
    return binary_string
    

In [111]:
total = 1
for thread in module_lib_backup['broadcaster']['destinations']:
    binary_string = get_thread_binary_representation(thread, get_accumulator(thread))
    numeric = binary_to_int(binary_string)
    print(binary_string, numeric)
    total *= numeric
total
    

['1', '0', '1', '1', '0', '1', '0', '1', '1', '1', '1', '1'] 4013
['1', '0', '0', '0', '0', '1', '0', '1', '1', '1', '1', '1'] 4001
['1', '1', '0', '1', '0', '0', '0', '0', '1', '1', '1', '1'] 3851
['1', '1', '1', '0', '0', '0', '1', '0', '1', '1', '1', '1'] 3911


241823802412393